In [1]:
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm
import requests 
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

C:\Users\Lenovo\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [16]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [3]:
mistral_key = "pqcHo388Jwjc76uZ3kBVhGCMpiobC3kF"
es_client = Elasticsearch('http://localhost:9200') 

In [ ]:
#create index settings
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

In [6]:
for doc in tqdm(documents):
    # input the data to elastic search
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:54<00:00, 17.48it/s]


In [12]:
def elastic_search(query):
    search_query = {
        "size": 5, # number of results
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text", "section"], # question boost 4
                        "type": "best_fields"
                    }
                }
                # "filter": {
                #     "term": {
                #         "course": filter
                #     }
                # }
            }
        }
    }

    response = es_client.search(index="course-questions", body=search_query)
    
    result_docs = []
    # print(response)
    for hit in response['hits']['hits']:
        # result_docs.append(hit['_source']),
        result_docs.append(hit)
    
    return result_docs

In [13]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['_source']['section']}\nquestion: {doc['_source']['question']}\nanswer: {doc['_source']['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [9]:
def llm(prompt, model="mistral-medium-latest"):
    messages = [
        ChatMessage(role="user", content=prompt)
    ]
    client = MistralClient(api_key=mistral_key)
    chat_response = client.chat(
        model=model,
        messages=messages
    )
    return (chat_response.choices[0].message.content)

In [10]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [14]:
query = 'How do I execute a command in a running docker container?'
result = rag(query)

print(result)

To execute a command in a running Docker container, you can use the `docker exec` command followed by `-it` for an interactive terminal, `<container-id>` for the ID of the container, and `bash` to start a bash shell. Here is an example:

```
docker exec -it <container-id> bash
```

You can find the container ID by running the `docker ps` command, which will list all running containers along with their IDs.

Note that this will start a new bash shell in the container, and you will need to navigate to the correct directory and run any additional commands manually. If you want to run a specific command directly, you can specify it after the `bash` command, like this:

```
docker exec -it <container-id> bash -c "ls /path/to/directory"
```

This will run the `ls` command in the `/path/to/directory` directory of the container and then exit the bash shell.

The context provided suggests that the `docker exec` command is used to execute a command in a running Docker container. It also mentions